In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import pandas as pd
import sqlalchemy as sa
import sys

from dataframe import preprocess, rename_header_dataframe

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import DataSP

## Database connection

In [3]:
engine, session = db.connect()
engine.echo = False

2023-03-24 19:57:55,558 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-24 19:57:55,559 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 19:57:55,561 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-24 19:57:55,561 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 19:57:55,562 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-24 19:57:55,563 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load CSV files

In [4]:
filename = '../csv/george_data.csv'
df = pd.read_csv(filename, sep=';', low_memory=False, skipinitialspace=True)

## Adds 'yes' in the George column ('yes' are records marked as reliable by George))

In [5]:
count_records_marked_by_george = session.query(DataSP) \
    .filter(DataSP.george == True) \
    .count()

if count_records_marked_by_george == 0:
    for row in df.iterrows():
        get_value_column_george = row[1]['GEORGE']
        if get_value_column_george.lower() == 'sim':
            session.query(DataSP) \
                .filter(DataSP.seq == row[1]['seq']) \
                .update({'george': True}, synchronize_session=False)
            session.commit()

    count_records_marked_by_george = session.query(DataSP) \
        .filter(DataSP.george == True) \
        .count()
    print('count of records in table %s: %d' % (DataSP.__tablename__, count_records_marked_by_george))

count of records in table data_sp: 1419


## Close connection

In [6]:
session.close()
engine.dispose()